# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).

## Environment Set-Up

In [38]:
import os
import glob
import tensorflow as tf
import numpy as np
#import problem_unittests as tests
from collections import Counter
import pickle

## Get the Data
The data is 10 scripts, found on the web.

In [39]:
def read_script(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [40]:
def load_all_scripts(directory):
    scripts = []
    for file_path in glob.glob(os.path.join(directory, '*.txt')):
        script_text = read_script(file_path)
        scripts.append(script_text)
    return scripts

In [41]:
def combine_scripts(scripts):
    combined_script = "\n\n".join(scripts)  # Adds two newline characters between scripts
    return combined_script

In [42]:
# Load and combine all scripts
scripts_directory = r'C:\Users\Rachel Montgomery\Documents\fall2023\ml2\simpsons\simpsons-script-generator\Data'
scripts = load_all_scripts(scripts_directory)
combined_script = "\n\n".join(scripts)  # Adds two newline characters between scripts

In [43]:
# Proceed with preprocessing 
text = combined_script[81:]  # Adjust as needed

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [44]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 14098
Number of scenes: 930
Average number of sentences in each scene: 9.386021505376345
Number of lines: 9659
Average number of words in each line: 5.608758670669841

The sentences 0 to 10:
long strand of sparkly white toothpaste swirls
around a running sink.  A strand of green, Prellstyle shampoo races alongside the toothpaste, and
both spiral towards the drain.
BART (O.S.)
Go, toothpaste!  Go!  Move
your pasty white butt!
LISA (O.S.)
Come on, shampoo!  You can
do it!
The shampoo reaches the center first and disappears


## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [45]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    words = text.split()
    word_counts = Counter(words)

    # Sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    # Create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

# Example usage of the function
sample_text = "hello world hello"
vocab_to_int, int_to_vocab = create_lookup_tables(sample_text)

print(vocab_to_int)  # Should show the mapping of words to integers
print(int_to_vocab)  # Should show the mapping of integers to words


{'hello': 0, 'world': 1}
{0: 'hello', 1: 'world'}


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [46]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    return {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_mark||',
        '?': '||Question_mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '--': '||Dash||',
        '\n': '||Return||'
    }

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [47]:
def tokenize_script(script, token_lookup):
    tokens = token_lookup()
    for key, token in tokens.items():
        script = script.replace(key, ' {} '.format(token))
    return script

# Assuming combined_script contains all your script data
tokenized_script = tokenize_script(combined_script, token_lookup)

In [48]:
# Create lookup tables
vocab_to_int, int_to_vocab = create_lookup_tables(tokenized_script)

# Convert your script to integers
int_script = [vocab_to_int[word] for word in tokenized_script.split()]

In [49]:
# Save the preprocessed data
def save_preprocessed_data(data, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(data, file)

# Choose file paths for saving
int_script_path = './preprocessed_script.pkl'
vocab_to_int_path = './vocab_to_int.pkl'
int_to_vocab_path = './int_to_vocab.pkl'

# Save the data
save_preprocessed_data(int_script, int_script_path)
save_preprocessed_data(vocab_to_int, vocab_to_int_path)
save_preprocessed_data(int_to_vocab, int_to_vocab_path)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [50]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'input')
    
    targets = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'targets')
    
    learning_rate = tf.placeholder(dtype = tf.float32, name = 'learning_rate')
    
    return (inputs, targets, learning_rate)


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [51]:
def get_init_cell(batch_size, rnn_size, keep_prob = 0.8):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units = rnn_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
    
    cell = tf.contrib.rnn.MultiRNNCell([drop]) 
    
    initial_state = cell.zero_state(batch_size = batch_size, dtype = tf.float32)
    
    initial_state = tf.identity(initial_state, name = "initial_state")
    
    return (cell, initial_state)


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [52]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform(shape = (vocab_size, embed_dim), minval = -1, maxval = 1))
    
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [53]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell = cell, inputs = inputs, dtype = tf.float32)
    final_state = tf.identity(final_state, name = 'final_state')
    return (outputs, final_state)

### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [54]:
import tensorflow as tf

# Define the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vocab_to_int), output_dim=256),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.Dense(len(vocab_to_int))
])


In [55]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


## Prepare Data for Training

In [56]:
# Example: Creating training sequences and targets
seq_length = 100  # length of each input sequence
X = []
y = []

for i in range(0, len(int_script) - seq_length):
    X.append(int_script[i:i+seq_length])
    y.append(int_script[i+seq_length])

X = np.array(X)
y = np.array(y)


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

In [57]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # Total number of batches
    total_bathces = int(len(int_text) / (batch_size * seq_length))
     
    # Making full batches
    X = np.array(int_text[: total_bathces * batch_size * seq_length])
    y = np.array(int_text[1: total_bathces * batch_size * seq_length + 1])
    
    y[-1] = X[0] # TODO: delete
    
    X_batches = np.split(X.reshape(batch_size, -1), total_bathces, 1)
    y_batches = np.split(y.reshape(batch_size, -1), total_bathces, 1)
    
    full_batches = np.array(list(zip(X_batches, y_batches)))
    
    # TODO: Implement Function
    return full_batches


### Train Model

In [58]:
# Define batch size and number of epochs
batch_size = 128
num_epochs = 20

# Train the model
model.fit(X, y, batch_size=batch_size, epochs=num_epochs)


Epoch 1/20
612/612 [==============================] - 3312s 5s/step - loss: 6.5919
Epoch 2/20
  8/612 [..............................] - ETA: 55:07 - loss: 6.2683

KeyboardInterrupt: 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [ ]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [ ]:

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [ ]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = tf.Graph.get_tensor_by_name(loaded_graph, name = "input:0")
    InitialStateTensor = tf.Graph.get_tensor_by_name(loaded_graph, name = "initial_state:0")
    FinalStateTensor = tf.Graph.get_tensor_by_name(loaded_graph, name = "final_state:0")
    ProbsTensor = tf.Graph.get_tensor_by_name(loaded_graph, name = "probs:0")
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)

tests.test_get_tensors(get_tensors)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    predicted_index = np.random.choice(a = len(int_to_vocab), p =probabilities)
    predicted_word = int_to_vocab[predicted_index]
    return predicted_word

tests.test_pick_word(pick_word)

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 100
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

In [ ]:
def generate_script(seed_word, gen_length=100):
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load the model
        loader = tf.train.import_meta_graph(load_dir + '.meta')
        loader.restore(sess, load_dir)

        # Get Tensors from the loaded model
        input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

        # Start the script with the seed word (character or setting)
        gen_sentences = [seed_word + ':']
        prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

        # Generate the script
        for n in range(gen_length):
            dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
            dyn_seq_length = len(dyn_input[0])

            probabilities, prev_state = sess.run(
                [probs, final_state],
                {input_text: dyn_input, initial_state: prev_state})

            pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
            gen_sentences.append(pred_word)

        # Format and return the script
        script = ' '.join(gen_sentences)
        for key, token in token_dict.items():
            script = script.replace(' ' + token.lower(), key)
        script = script.replace('\n ', '\n').replace('( ', '(')
        return script

# Example usage
print(generate_script('homer_simpson'))


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.